# Practicando con el método de los mínimos cuadrados en Python
En este notebook aprenderás a ejecutar el método de los mínimos cuadrados, disponible en las librerías *statsmodels* y *scikit-learn*. Veremos cómo cargar un conjunto de datos y seleccionar las variables con las construir el modelo de regresión lineal más conocido.
## Método de los mínimos cuadrados en ***statsmodels***
En la primera parte del *notebook* utilizaremos la librería *statsmodels*. Es importante familiarse con su [API](https://www.statsmodels.org/stable/api.html) para conocer los métodos disponibles y sus estructuras de datos. Lo primero que debemos hacer es importar los paquetes que vamos a necesitar:

In [1]:
import numpy as np
import statsmodels.api as sm

### Ejemplo sobre una muestra de datos aleatoria
En primer lugar, vamos a generar una muestra de observaciones (*x*,*y*). Los valores de *x* van a variar entre 0 y un tamaño de muestra dado. Los valores de *y* queremos que sean dependientes de *x*, así que los generamos a partir de *x* añadiendo cierto ruido. 

In [2]:
tam_muestra = 50
x = np.arange(0, tam_muestra, step=1)
y = x*2 + 3 + np.random.normal(0,1,tam_muestra)
print(y)

[  1.61964986   5.26944421   7.59556427   6.78527242  10.91419013
  12.30382667  16.36289948  16.13243057  18.3076106   20.34590626
  23.69923599  26.8251084   28.15906321  29.05713008  31.62515406
  32.23228054  34.6396282   38.30772843  39.83940959  41.61280664
  42.79823959  45.04424978  46.55430776  48.66059686  51.02033464
  54.57313624  54.39727081  58.11335177  57.34724364  60.41823025
  62.40925055  63.97055989  66.24035985  68.92807263  72.514028
  74.40802475  72.87255868  76.52895483  80.10399199  81.31049164
  83.45930531  87.47090272  86.90376828  88.98708868  90.61014641
  95.02781652  95.78483876  96.87353198  99.13938222 100.44685287]


A partir de esta muestra de datos ya tenemos lo necesario para realizar un ajuste por medio del método de los mínimos cuadrados. En *statsmodels*, este método se denomina [OLS](https://www.statsmodels.org/stable/examples/notebooks/generated/ols.html#) (*Ordinary Least Squares*). Primero tenemos que crear un objeto de la clase. Para realizar el ajuste, invocamos a la función *fit*, que nos devuelve el método entrenado:

In [3]:
algoritmo = sm.OLS(endog=y,exog=x)
modelo = algoritmo.fit()

La expresión con la que generamos los valores *y* se asemeja a la curva de regresión lineal con pendiente igual 2 e intercepto igual a 3. Para ver los coeficientes estimados por OLS, accedemos a la propiedad *.params*

In [4]:
modelo.params

array([2.09629178])

Vemos que solo se ha estimado un coeficiente, cercano al valor real de la pendiente. Por defecto, OLS no estima el intercepto salvo que se lo indiquemos expresamente con la función *add_constant*:

In [5]:
x = sm.add_constant(x)
algoritmo = sm.OLS(y,x)
modelo_con_intercepto = algoritmo.fit()
intercepto = modelo_con_intercepto.params[0]
print(intercepto)
pendiente = modelo_con_intercepto.params[1]
print(pendiente)

2.841400313043476
2.0101887443792674


### Ejemplo sobre un conjunto de datos
En este apartado vamos a realizar el mismo procedimiento pero utilizando variables tomadas de un conjunto de datos. La librería *statsmodels* nos permite cargar *datasets* directamente, aunque también podríamos utilizar un fichero CSV e importarlo con *Pandas*. Vamos a utilizar un *dataset* de la categoría *HistData* que contiene datos sobre las estaturas de padres e hijos.

In [6]:
# Importar el conjunto de datos
dataset = sm.datasets.get_rdataset("GaltonFamilies", "HistData")
# Ver la información del conjunto de datos (10 primeras filas)
print(dataset.data.head(10))
# Obtener los valores de las variables de interés
altura_padres = dataset.data.get('midparentHeight')
altura_hijos = dataset.data.get('childHeight')

  family  father  mother  midparentHeight  children  childNum  gender  \
0    001    78.5    67.0            75.43         4         1    male   
1    001    78.5    67.0            75.43         4         2  female   
2    001    78.5    67.0            75.43         4         3  female   
3    001    78.5    67.0            75.43         4         4  female   
4    002    75.5    66.5            73.66         4         1    male   
5    002    75.5    66.5            73.66         4         2    male   
6    002    75.5    66.5            73.66         4         3  female   
7    002    75.5    66.5            73.66         4         4  female   
8    003    75.0    64.0            72.06         2         1    male   
9    003    75.0    64.0            72.06         2         2  female   

   childHeight  
0         73.2  
1         69.2  
2         69.0  
3         69.0  
4         73.5  
5         72.5  
6         65.5  
7         65.5  
8         71.0  
9         68.0  


Vamos a construir un modelo de regresión que establezca la relación entre la altura de los padres y los hijos. Utilizaremos solo una parte de los datos, para estudiar con el resto cómo de buena es la predicción respecto al valor real.

In [7]:
altura_padres = sm.add_constant(altura_padres)
algoritmo = sm.OLS(altura_hijos,altura_padres)
modelo_altura = algoritmo.fit()
print(modelo_altura.params)

const              22.636241
midparentHeight     0.637361
dtype: float64


El modelo de regresión nos permite extraer los valores estimados de *y*, por lo que podemos calcular nosotros mismos los residuos:

In [8]:
altura_hijos_pred = modelo_altura.fittedvalues
residuos = np.asarray(altura_hijos - altura_hijos_pred)
print(residuos)

[ 2.48762699e+00 -1.51237301e+00 -1.71237301e+00 -1.71237301e+00
  3.91575578e+00  2.91575578e+00 -4.08424422e+00 -4.08424422e+00
  2.43553321e+00 -5.64466785e-01  1.93553321e+00 -6.44667852e-02
 -1.56446679e+00 -4.06446679e+00 -5.56446679e+00  5.32849508e+00
  2.32849508e+00  1.32849508e+00 -1.71504921e-01 -4.17150492e+00
 -4.17150492e+00 -1.22485874e-01  6.87751413e+00  4.37751413e+00
  3.37751413e+00  3.37751413e+00  8.77514126e-01 -5.62248587e+00
  1.39377645e+00 -1.10622355e+00 -3.10622355e+00 -3.09347633e+00
 -3.26204866e+00  6.44256343e+00  2.44256343e+00  4.42563432e-01
 -5.57436568e-01 -5.57436568e-01 -1.55743657e+00 -4.05743657e+00
 -4.55743657e+00 -2.21326168e+00  2.04036946e+00 -6.95963054e+00
 -9.59630541e-01 -1.95963054e+00  2.21245690e+00  1.71245690e+00
 -2.08754310e+00  3.72871923e+00  2.22871923e+00  1.92871923e+00
  1.92871923e+00  9.28719227e-01  4.28719227e-01 -1.77128077e+00
 -3.77128077e+00 -4.77128077e+00  5.90080667e+00  4.90080667e+00
  3.40080667e+00 -5.59919

## Método de los mínimos cuadrados en ***scikit-learn***
En la segunda parte del *notebook* utilizaremos la librería *scikit-learn*. Al igual que con *statsmodels*, es importante saber consultar su [API](https://scikit-learn.org/stable/modules/classes.html) para familiarizarse con sus clases y comprender los parámetros que requiere cada método. En concreto, en este *notebook* vamos a utilizar el paquete de regresión, donde está implementado el método de los mínimos cuadrados (denominado simplemente *Linear Regression*). También importamos el paquete con conjuntos de datos precargados.

In [9]:
from sklearn.linear_model import LinearRegression
import sklearn.datasets as skldata 

En primer lugar, vamos a importar el conjunto de datos *Boston*, cuya variable independiente representa el precio de viviendas. El objeto devuelto es un diccionario que contiene, además de los datos, el nombre de las variables, una descripción, etc.

In [10]:
datos_viviendas = skldata.load_boston()
print(datos_viviendas['feature_names'])
print(datos_viviendas['DESCR'])


['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial hig

Otra forma de cargar los datos es indicar, mediante el parámetro *return_X_y*, que nos devuelva únicamente los valores separando las variables dependientes (*X*) de la variable independiente (*y*). Para ajustar un modelo de regresión simple con una variable, nos quedaremos con una única variable dependiente (DIST, distancia a centros de empleo).

In [11]:
X, y = skldata.load_boston(return_X_y=True)
x = X[:, np.newaxis, 7]

En este caso, las variables no representan el mismo concepto como ocurría con el ejemplo de las alturas de padres e hijos. Si tenemos variables con distintas unidades de medida, es conveniente escalar los valores para que el modelo no se vea afectado por los rangos de las variables. La librería *scikit-learn* nos ofrece clases para realizar este tipo de preprocesado.

In [12]:
from sklearn.preprocessing import StandardScaler
# Escalar con media = 0 y desviación estándar = 1
alg_escalado = StandardScaler(with_mean=True, with_std=True)
x_escalado = alg_escalado.fit_transform(x)
# Para la variable y, los valores tienen que ser convertidos antes a un array de 2 dimensiones
y_escalado = alg_escalado.fit_transform(y.reshape(-1,1))
print(x_escalado[0:10])
print(y_escalado[0:10])

[[0.1402136 ]
 [0.55715988]
 [0.55715988]
 [1.07773662]
 [1.07773662]
 [1.07773662]
 [0.83924392]
 [1.02463789]
 [1.08719646]
 [1.32963473]]
[[ 0.15968566]
 [-0.10152429]
 [ 1.32424667]
 [ 1.18275795]
 [ 1.48750288]
 [ 0.6712218 ]
 [ 0.03996443]
 [ 0.49708184]
 [-0.65659542]
 [-0.39538548]]


Vamos a separar los valores en dos muestras, una para entrenamiento (*train*) y otra para test, de forma que podamos utilizar el modelo para predicir sobre datos no vistos. Para ello, utilizamos la función *train_test_split*, indicándole que queremos reservar un 33% de los datos para test.

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

Ahora podemos invocar al método de los mínimos cuadrados para ajustar un modelo lineal entre el precio de la vivienda y la distancia a los centros de empleo:

In [14]:
alg_regresion = LinearRegression()
modelo_precio = alg_regresion.fit(x_train, y_train)
print(modelo_precio)

LinearRegression()


Podemos extraer los valores de los coeficientes estimados accediendo a las propiedades del modelo:

In [15]:
intercepto = modelo_precio.intercept_
print(intercepto)
pendiente = modelo_precio.coef_[0]
print(pendiente)

17.42376590641694
1.2608570184853074


Con la función *predict*, calculamos los valores estimados de *y* para la partición de test. A continuación, podemos calcular los residuos puesto que tenemos guardados los valores reales para esa partición.

In [16]:
y_pred = modelo_precio.predict(x_test)
residuos = y_test - y_pred
print(residuos)

[-6.90295220e-01 -3.76300748e-01  1.20685860e+01 -4.02459183e+00
  1.59518703e-02 -1.47539170e-01 -3.63041959e+00  9.46158887e-01
 -1.23277310e+00  2.02354509e+00  1.05869522e+01 -4.87639798e+00
  1.48532786e+00 -7.23806818e+00 -5.69948518e+00 -4.63722640e+00
  1.13634173e+01 -6.68739629e+00 -8.82543076e+00 -1.59703470e+00
  1.56417276e+00 -3.85464104e+00 -8.08722757e+00 -4.79350248e+00
 -2.29955667e+00 -3.02532027e+00  2.89268095e+01 -5.02953569e+00
  6.56928664e+00 -5.56045607e+00  2.40292319e+00  3.11021662e+01
 -1.53476136e+00 -8.86428075e+00  2.85425003e+01 -5.95668753e+00
  3.00191158e-01  2.01261775e+00  4.07325093e+00  1.11753509e+00
  1.89202250e+01  1.25058377e+01  7.82639099e+00  1.62360202e+00
  1.96299718e+00  1.61874144e+01 -3.93934386e+00 -2.94969667e+00
 -8.97143982e+00 -4.49126205e+00  3.27595515e+00 -7.99728583e+00
 -2.54582908e+00 -6.56402676e+00 -2.04431683e+00 -2.48653969e+00
 -8.18784396e+00 -4.79126205e+00 -7.03890188e-02 -8.66433612e+00
 -2.05793461e+00  7.96063